## 5.2 Abandoned Lock
In the previous example, the critical section for this program exists between the acquire methods on lines 14 and 15, and the release methods on lines 21 and 22. If one of the philosopher's threads acquires the locks, and when something goes wrong in that critical section to cause an unexpected error, that could kill its thread before it gets a chance to release the lock. To simulate happening, We'll add another `if` statement to check it there're exactly 10 pieces of sushi left and do my favorite technique for intentionally crashing a program - divide by 0. 

We should never divide by 0, but we're doing it here to trigger an exception that will cause Python to crash the currently executing thread.

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()

        if sushi_count > 0:
            sushi_count -= 1
            print(name, 'took a piece! Sushi remaining:', sushi_count)
            
        if sushi_count == 10:
            print(1/0)

        second_chopstick.release()
        first_chopstick.release()
        
if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()

If we run this program, it will get all the way down to 10 pieces remaining. Then the thread that happens to be executing at that time, in this case, that's Olivia thread, it hits that divide by zero and crashes. The other threads are stuck waiting on the locks that will never get released by Olivia, so the program is stuck here forever. 

This scenario is not the same as deadlock, because the threads here are not waiting on each other to release a lock. But it's a related scenario and the impact is the same: the program isn't making any progress. 


To prevent this type of situation from occurring, we should put the critical section within a `try` block. If we have any exception handling code, we can optionally include an `except` clause after the `try` block to catch and deal with that error. But what we really care about here is making sure that the locks always get released before the current thread gets terminated if it crashes. To do that, We'll also add a `finally` block after the `try` statement, and put the calls to unlock the chopsticks in it.

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()

        try:
            if sushi_count > 0:
                sushi_count -= 1
                print(name, 'took a piece! Sushi remaining:', sushi_count)

            if sushi_count == 10:
                print(1/0)

        finally:
            second_chopstick.release()
            first_chopstick.release()
        
if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()


This time, an `exception` still occurs when one of the threads takes the 10th piece of sushi, in this case, that was Olivia, but thanks to the `finally` clause, that thread is able to release the lock before it terminates. We can see that after the Olivia thread took the 10th piece of sushi and crashed, the Barron thread took over to finish eating the remaining sushi.

It's good practice to always make sure lock will be released if something goes wrong and unexpectedly crashes a thread. Python makes that especially easy because `lock` objects support working with context managers. Using the `with` statement on a lock is equivalent to using the `try` and `finally` blocks. Using a context manager is a more pythonic way to program.

```
some_lock.acquire()
try:
    # do something...
finally:
    some_lock.release()
    
# More pythonic structured
with some_lock:
    # do something...
```

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)

                if sushi_count == 10:
                    print(1/0)

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()
